<a href="https://colab.research.google.com/github/anskong/ai_ref_from_lessons/blob/main/LGCNS_LangChain_012.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텍스트 분할기를 이용한 텍스트 분할
- PDF문서를 업로드하여 추출
- 예제 문서로더를 사용하여 텍스트 추출
- 다운로드 샘플문서(2024 한국인터넷백서): https://www.nia.or.kr/site/nia_kor/ex/bbs/View.do?cbIdx=99871&bcIdx=27866&parentSeq=27866

In [ ]:
# 1) 필수 패키지 설치
!pip install --quiet --upgrade "langchain-core>=0.3.0" "langchain-community>=0.0.30" "langchain-text-splitters>=0.0.1" pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.3/438.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


In [ ]:
# 2) Colab 파일 업로드
from google.colab import files
import os

print("📤 PDF 파일을 업로드하세요...")
uploaded = files.upload()
filename = list(uploaded.keys())[0]  # 업로드된 파일명

# 3) LangChain 로더 및 스플리터 임포트
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 4) PDF 로드 및 페이지 단위 분할
loader = PyPDFLoader(filename)
pages = loader.load()

# 5) 텍스트 분할기 정의 (청크 크기와 중첩 조절 가능)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=400
)

# 6) 분할 수행
splits = splitter.split_documents(pages)

# 7) 결과 출력
print(f"📄 총 청크 수: {len(splits)}")

n = 0  # 예시: 첫 번째 청크 미리보기
print(f"\n📑 청크 {n + 1} 내용:\n")
print(splits[n].page_content[:500])

📤 PDF 파일을 업로드하세요...


Saving 2024_한국인터넷백서_펼침.pdf to 2024_한국인터넷백서_펼침.pdf
📄 총 청크 수: 155

📑 청크 1 내용:

한국인터넷백서
2024
KOREA INTERNET WHITE PAPER
KOREA INTERNET WHITE PAPER
보이스아이 앱을 활용하여
바코드를 인식하면 
2024 한국인터넷백서를 
음성으로 들으실 수 있습니다.


# 내장모델을 이용한 텍스트 벡터화 (PDF 업로드 + 텍스트 분할 + 임베딩 벡터화)


In [ ]:
# 1) 필수 패키지 설치
!pip install --quiet --upgrade "langchain-core>=0.3.0" "langchain-community>=0.0.30" "langchain-text-splitters>=0.0.1" "langchain-openai>=0.0.8" openai pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.4/720.4 kB 25.3 MB/s eta 0:00:00


In [ ]:
# 2) Colab 파일 업로드
from google.colab import files
import os
from google.colab import userdata

# API 키 등록
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print("📤 PDF 파일을 업로드하세요...")
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 3) LangChain 모듈 불러오기
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

# 4) PDF 로드 및 텍스트 분할
loader = PyPDFLoader(filename)
pages = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=400
)
splits = splitter.split_documents(pages)

# 5) 임베딩 벡터화 수행 (text-embedding-3-small)
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_API_KEY
)

# 예시로 11번째 청크 벡터화
content = splits[10].page_content
vector = embeddings.embed_query(content)

# 6) 결과 출력
print(f"✅ 내장 벡터 차원수: {len(vector)}")
print(f"🔍 내장 벡터 처음 10개 요소: {vector[:10]}")

📤 PDF 파일을 업로드하세요...


Saving 2024_한국인터넷백서_펼침.pdf to 2024_한국인터넷백서_펼침 (1).pdf
✅ 내장 벡터 차원수: 1536
🔍 내장 벡터 처음 10개 요소: [0.01433388702571392, 0.02328975312411785, 0.013310037553310394, 0.0182492658495903, 0.1010797768831253, -0.029320336878299713, -0.05351017788052559, 0.08442816883325577, 0.012533712200820446, 0.03593597561120987]
